# Защита персональных данных 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
print(df.duplicated().sum()) 

153


In [6]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Всего 5000 строк и 5 столбцов. Пропусков нет. Есть неверный тип данных в столбцах. Надо преобразовать float в int.
Также есть дубликаты. Возможно просто совпадение. В связи с тем, что данные чистые, скорее всего просто совпдает возраст с заработной платой. 

In [7]:
# преобразовываем тип данных
df['Возраст'] = df['Возраст'].astype(int)
df['Зарплата'] = df['Зарплата'].astype(int)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int64
 1   Возраст            5000 non-null   int64
 2   Зарплата           5000 non-null   int64
 3   Члены семьи        5000 non-null   int64
 4   Страховые выплаты  5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


In [9]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


### Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)


Обозначения:
- X - матрица признаков (нулевой столбец состоит из единиц)
- y - вектор целевого признака
- P - обратимая квадратная матрица (матрица преобразования)
- w - вектор весов линейной регрессии (нулевой элемент равен сдвигу)
- a - вектор предсказаний 
- w’ - вектор весов после умножения на обратимую матрицу
- a’ - вектор предсказаний после умножения на обратимую матрицу. 

Формула предсказаний:
$$
a = Xw
$$

Задача обучения:
$$
w = \arg\min_w MSE(Xw, y)
$$

Исходное уравнение весов:
$$
w = (X^{T}X)^{-1}X^{T}y
$$

Формулы:
- $AB \neq BA$
- $ (AB)^{-1} = B^{-1}A^{-1} $
- $ (AB)^{T} = B^{T}A^{T} $
- $ AA^{-1} = E $
- $ АЕ = ЕА = А $

Запишем вместо Х произведение Х\*Р и w’ вместо w.

$ w'=((XP)^{T}(XP))^{-1}(XP)^{T}y = (P^{T}X^{T}XP)^{-1}P^{T}X^{T}y = P^{-1}(X^{T}X)^{-1}P^{T^{-1}}P^{T}X^{T}y$, где $P^{T^{-1}}P^{T} = E$, тогда $w'=P^{-1}*[(X^{T}X)^{-1}X^{T}y]=P^{-1}w$ 

связь между $w$ и $w'$ мы нашли, теперь подставим в 
$$
a = Xw
$$ 

Получаем: $a'= XP*w' = XPP^{-1}w = XEw = Xw = a$

Таким образом получается, что если признаки умножить на обратимую матрицу, то качество линейной регрессии не изменится. 

Соотвественно, чтобы защитить данные клиентов страховой компании, можно преобразовать данные с помощью квадратной обратимой матрицы.

In [10]:
# проверим гипотезу на случайных матрицах
np.random.seed(17894)
check = []
for i in range(1200):
    P = np.random.normal(10, 200, size=(3,3))
    A = np.random.normal(70, 500, size=(3,3))
    check.append(np.linalg.det(np.rint(A @ P @ np.linalg.inv(P)) == A))
print(P)
print()
print(A)
np.prod(np.array(check))

[[-144.96202191   16.26483752  234.48403305]
 [  35.00979287   -2.54540249  -23.23419995]
 [ -67.27941477  257.28584219   27.59253369]]

[[  82.80729122  968.76080405  598.82586486]
 [-454.5420903  -163.42369248  -27.4071581 ]
 [  93.22118729 -352.57639869  240.97380128]]


0.0

## Алгоритм преобразования

**Алгоритм**

Для шифрования воспользуемся квадратной обратимой матрицей - $P$, где $XP=X_{cripto}$ - процесс шифрования.

Создадим случайную квадратную обратимую матрицу и умножим на нее признаки. Сравним качество линейной регрессии до шифрования и после. 

## Проверка алгоритма

In [11]:
X = df.drop('Страховые выплаты', axis = 1)
Y = df['Страховые выплаты']
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=12345)

In [12]:
class LinearRegression:
    def fit(self, f_train, t_train):
        X = np.concatenate((np.ones((f_train.shape[0], 1)), f_train), axis=1)
        y = t_train
        w = (np.linalg.inv(X.T.dot(X)).dot(X.T)).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, f_test):
        return f_test.dot(self.w) + self.w0

In [13]:
model = LinearRegression()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print(r2_score(y_test, predictions))

0.43522756840833365


In [14]:
model = LinearRegression()
model.fit(X, Y)
predictions = pd.Series(model.predict(X))
r2_without_cripto = r2_score(Y, predictions)

#Модель для шифрования
model_cripto = LinearRegression()

#Шифруем
loc = np.random.randint(2,100)
std = np.random.randint(2,100)
P = np.random.normal(loc, std, size=(4,4)).astype('int64')
X_cripto = pd.DataFrame(data = X.values @ P, columns = X.columns)

#Проверка на шифрованных данных
model_cripto.fit(X_cripto, Y)
predictions = pd.Series(model_cripto.predict(X_cripto))
r2_with_cripto = r2_score(Y, predictions)

print(r2_without_cripto)
print(r2_with_cripto)

0.42494550308169177
0.4249455030816731


In [15]:
# проверим созданную матрицу P на обратимость
P_inv = np.linalg.inv(P)
print(P_inv)

[[-0.00814682  0.08995558 -0.03759981  0.01325129]
 [ 0.01640759 -0.06021253  0.03304891 -0.02740318]
 [ 0.00838055 -0.01489192 -0.00908432  0.02467419]
 [-0.00283582 -0.02431809  0.02625072 -0.00031466]]


In [16]:
X_cripto.head()

,Пол,Возраст,Зарплата,Члены семьи
0,1588413,1091918,396998,2927302
1,1217298,836731,304168,2242997
2,672812,462464,168087,1239609
3,1335008,917726,333723,2460803
4,836039,574715,208929,1540498


## Вывод
Данные были загружены и изучены. Не было обнаружено пропусков, однако были дубликаты, скорее всего связаны со схожестью информации по клиентам. 
Проверена и доказана гипотеза о том, что если признаки умножить на обратимую матрицу, то качество линейной регрессии не изменится. 
Был предложен алгоритм преобразования данных для защиты персональной информации. Алгоритм представляет собой преобразование данных за счет умножения на обратимую квадратную матрицу. 